In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from sparsesvd import sparsesvd 
from scipy.sparse import csc_matrix

In [3]:
data = pd.read_csv('data/data.txt', sep=' ', header = None)
X = np.array(data.T)

In [9]:
def argminBIC(lambdas, w_inv_nonzero, cand_nonzero, index, X, sigma_sq, n, d, old, isV):
    """index for lowest BIC"""
    BICs = np.ones(lambdas.shape[0]-1)*np.Inf
    for i in range(BICs.shape[0]):
        temp_partial = update(cand_nonzero, lambdas[i])/w_inv_nonzero
        if(isV==1):
            temp = np.zeros((d,1))
            temp[index] = temp_partial
            pred = old@temp.T
        else:
            temp = np.zeros((n,1))
            temp[index] = temp_partial
            pred = temp@old.T
        BICs[i] = np.sum((X-pred)**2)/sigma_sq + np.sum(temp_partial!=0)*np.log(n*d)
    best = np.argmin(BICs)
    return best

In [4]:
def update(candidate, penalty):
    """update v or u for SSVD"""
    return np.sign(candidate)*(np.abs(candidate)>= penalty)*(np.abs(candidate) - penalty)

In [22]:
def SSVD_primary(X, gamma1 = 2, gamma2 = 2, tol = 1e-4, max_iter = 100):
    """SSVD for 1 layer"""
    u, s, v = sparsesvd(csc_matrix(X), k=1)

    # initiations
    n = X.shape[0]
    d = X.shape[1]
    u = u.reshape((n,1))
    v = v.reshape((d,1))
    u_delta = 1
    v_delta = 1
    niter = 0
    SST = np.sum(X**2)
    
    while((u_delta > tol) or (v_delta > tol)):
        niter += 1

        ## Update v
        Xu = X.T @ u
        w2_inv = np.abs(Xu)**gamma2
        sigma_sq = np.abs(SST - sum(Xu**2))/(n*d-d)   #np.trace((X-s*u@v.T) @ (X-s*u@v.T).T)/(n*d-d)

        # prepare lambda and candicates
        Xu_w = Xu*w2_inv                                  # X.T @ u/w
        lambda2s = np.unique(np.append(np.abs(Xu_w), 0))
        lambda2s.sort()                                   # possible lambda2/2
        index = np.where(w2_inv>1e-8)
        w2_inv_nonzero = w2_inv[index]
        Xu_w_nonzero = Xu_w[index]

        # best lambda and new v
        best = argminBIC(lambda2s, w2_inv_nonzero, Xu_w_nonzero, index, X, sigma_sq, n, d, old=u, isV=1)
        lambda2 = lambda2s[best]
        v_new_partial = update(Xu_w_nonzero, lambda2)/w2_inv_nonzero
        v_new = np.zeros((d,1))
        v_new[index] = v_new_partial
        v_new = v_new/np.sqrt(np.sum(v_new**2))

        # update v
        v_delta = np.sqrt(np.sum((v-v_new)**2))
        v = v_new

        ## Update u
        Xv = X @ v
        w1_inv = np.abs(Xv)**gamma1
        sigma_sq = np.abs(SST - sum(Xu**2))/(n*d-n)   #np.trace((X-s*u@v.T) @ (X-s*u@v.T).T)/(n*d-d)

        # prepare lambda and candicates
        Xv_w = Xv*w1_inv                                  # X.T @ u/w
        lambda1s = np.unique(np.append(np.abs(Xv_w), 0))
        lambda1s.sort()                                   # possible lambda2/2
        index = np.where(w1_inv>1e-8)
        w1_inv_nonzero = w1_inv[index]
        Xv_w_nonzero = Xv_w[index]

        # best lambda and new u
        best = argminBIC(lambda1s, w1_inv_nonzero, Xv_w_nonzero, index, X, sigma_sq, n, d, old=v, isV=0)
        lambda1 = lambda1s[best]
        u_new_partial = update(Xv_w_nonzero, lambda1)/w1_inv_nonzero
        u_new = np.zeros((n,1))
        u_new[index] = u_new_partial
        u_new = u_new/np.sqrt(np.sum(u_new**2))

        # update u
        u_delta = np.sqrt(np.sum((u-u_new)**2))
        u = u_new
        
        # check iteration
        if(niter > max_iter):
            print("Fail to converge")
        
    return(u, v, niter)

In [23]:
result = SSVD_primary(X)

In [30]:
result[2]

6